# Building (a) search engine(s) with Whoosh and Annoy

Once we have our data (produced by CoronaWhy Team for Covid19 Kaggle link), we can read them in and feed both into a simple search index of Whoosh and into Annoy's search forest (link). 

Whoosh is a Python pure index search engine using Okapi BM25F ranking function as well other user-defined search functions. We use it here to perform a basic word-based search.

Annoy is a library for search in n-dimensional numerical space, e.g. word/document embeddings.

The results from the indexing modules can be combined in different way: with Whoosh as a n-gram filter and Annoy as refinement, with two competing search methods etc. 

In [ ]:
# -*- coding: utf-8 -*-
import json, os, spacy, re, gensim, string, collections, pickle, sys, time
from gensim.test.utils import common_texts
from gensim.corpora.dictionary import Dictionary
from gensim import corpora
import pandas as pd
import numpy as np

from pathos.helpers import cpu_count, freeze_support
from pathos.multiprocessing import ProcessingPool
from tqdm import tqdm

from whoosh.index import create_in
from whoosh.fields import *
from whoosh.qparser import QueryParser, OrGroup, MultifieldParser

from annoy import AnnoyIndex

spacy_nlp = spacy.load('en_core_sci_lg')
spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS
cpu_number = cpu_count()

import warnings
warnings.filterwarnings("ignore")

In [ ]:
source_folder = "..."

list_paths_source_texts = [os.path.join(source_folder,p) for p in os.listdir(source_folder)]
